In [6]:
import pandas as pd
import os
import numpy as np
from utils.data_process import sample_data

In [7]:
sample_size = 150

In [8]:
x = []
y = []
# if True:
for file in os.listdir("data/preds"):
    df = pd.read_csv(f"data/preds/{file}")
    pulse = np.array(df["pulse"])
    pulse, num_samples = sample_data(pulse)
    x.extend(pulse)
    y.extend([[df["sys"][0], df["dia"][0]]]*num_samples)
  
x = np.array(x)
y = np.array(y)
print(np.shape(x))
print(np.shape(y))


(274, 150)
(274, 2)


In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05, random_state=10)


In [15]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(150, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
# model.add(LSTM(64, return_sequences=True))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(2))  # 2 outputs for systolic and diastolic pressure

model.compile(optimizer='adam', loss='mse')


In [16]:
epochs = 20
batch_size = 35
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/20
8/8 [==============================] - 4s 27ms/step - loss: 11106.0537
Epoch 2/20
8/8 [==============================] - 0s 16ms/step - loss: 10761.6934
Epoch 3/20
8/8 [==============================] - 0s 23ms/step - loss: 9772.0742
Epoch 4/20
8/8 [==============================] - 0s 11ms/step - loss: 7621.3638
Epoch 5/20
8/8 [==============================] - 0s 13ms/step - loss: 4209.0581
Epoch 6/20
8/8 [==============================] - 0s 20ms/step - loss: 1345.7023
Epoch 7/20
8/8 [==============================] - 0s 12ms/step - loss: 1150.7067
Epoch 8/20
8/8 [==============================] - 0s 19ms/step - loss: 826.8116
Epoch 9/20
8/8 [==============================] - 0s 14ms/step - loss: 796.3251
Epoch 10/20
8/8 [==============================] - 0s 13ms/step - loss: 707.6251
Epoch 11/20
8/8 [==============================] - 0s 13ms/step - loss: 652.8559
Epoch 12/20
8/8 [==============================] - 0s 14ms/step - loss: 618.6122
Epoch 13/20
8/8 [===========

In [17]:
from keras.models import load_model
model2 = load_model("model2.keras")
y_pred = model.predict(x_test)
y2_pred = model2.predict(x_test)


1/1 [==============================] - 1s 1s/step


In [18]:
for i, val in enumerate(y_pred):
    print(f"{val} {y2_pred[i]} {y_test[i]}")

[117.27187   72.661835] [117.618935  79.29597 ] [115  81]
[127.217636  78.0099  ] [119.28988  80.59967] [114  76]
[126.96138  78.89867] [118.19662  79.65639] [153 100]
[101.30541   63.146797] [114.72007  77.28016] [101  67]
[144.54872  88.84439] [119.83663  80.73833] [137  89]
[104.33805  64.72723] [115.59908  77.87202] [127  89]
[129.61798  78.79413] [119.791115  81.04156 ] [118  79]
[108.30537  67.62474] [115.394844  77.69853 ] [115  81]
[113.78987  70.07536] [117.637886  79.378365] [101  67]
[146.01343  89.34703] [119.792305  80.71377 ] [114  76]
[137.8432   83.12146] [119.8751   80.95455] [134  87]
[124.68395  77.02475] [117.79713  79.39739] [115  77]
[122.43201  75.94113] [117.31926  79.00987] [162 100]
[122.60399  76.37425] [116.71288  78.53633] [136  85]


In [19]:
model.save("model3.keras")